# Spark assignment

Taratukhin Oleg, C41323

Initial env setup:

In [8]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pathlib import Path

spark = SparkSession.builder \
    .appName("itmo spark assignment") \
    .master("local[*]") \
    .getOrCreate()

spark

In [3]:
!cd data && ls
!rm -rf results/task3/

PATH_TO_DATA = Path("data/")
PATH_TO_RESULTS = Path("results/task3/")

followers.parquet              posts_api.json
followers_posts_api_final.json posts_likes.parquet
followers_posts_likes.parquet


Utilities:

In [9]:
def save_as_json(df: pyspark.sql.dataframe.DataFrame, name: str):
    df.write \
        .format("json") \
        .save(str(PATH_TO_RESULTS / name))

In [6]:
followers_posts = spark \
    .read \
    .json(str(PATH_TO_DATA / "followers_posts_api_final.json"))

followers_posts.limit(10).toPandas()

,attachments,comments,copy_history,copyright,date,final_post,from_id,geo,id,is_pinned,key,likes,owner_id,post_source,post_type,reposts,signer_id,text,unavailable,views
0,"[(None, None, None, None, None, None, None, No...","(0, 0, True)",None,None,1550165023,None,87449,None,3316,None,87449_3316,"(1, 1, 6, 0)",87449,"(None, None, iphone, api, None)",post,"(0, 0)",None,"Я люблю Вас. Я вчера, даже, хотел Вам это сказ...",None,"(428,)"
1,"[(None, None, None, None, None, None, (fe02668...","(0, 0, True)",None,None,1553774858,None,87449,None,3318,None,87449_3318,"(1, 1, 11, 0)",87449,"(None, None, iphone, api, None)",post,"(0, 0)",None,call me by your name ♡,None,"(477,)"
2,"[(None, None, None, None, None, None, (c25cda8...","(0, 0, True)",None,None,1555602008,None,87449,None,3320,None,87449_3320,"(1, 1, 10, 0)",87449,"(None, None, iphone, api, None)",post,"(0, 0)",None,🦋,None,"(419,)"
3,"[(None, None, None, None, None, None, None, No...","(0, 0, True)",None,None,1558105050,None,87449,None,3322,None,87449_3322,"(1, 1, 8, 0)",87449,"(None, None, iphone, api, None)",post,"(1, 0)",None,"Браво,Юра !",None,"(442,)"
4,"[(None, None, None, None, None, None, (7758fb4...","(0, 0, True)",None,None,1559301729,None,87449,None,3324,None,87449_3324,"(1, 1, 8, 0)",87449,"(None, None, iphone, api, None)",post,"(0, 0)",None,🕊,None,"(425,)"
5,"[(None, None, None, None, None, None, (666dd69...","(0, 0, True)",None,None,1564088944,None,87449,None,3325,None,87449_3325,"(1, 1, 10, 0)",87449,"(None, None, iphone, api, None)",post,"(0, 0)",None,Самый неприятный побочный эффект взросления дл...,None,"(301,)"
6,"[(None, None, None, None, None, None, (06ab977...","(0, 0, True)",None,None,1565162341,None,87449,None,3326,None,87449_3326,"(1, 1, 6, 0)",87449,"(None, None, iphone, api, None)",post,"(0, 0)",None,🔅,None,"(309,)"
7,"[(None, None, None, None, None, None, (74fdfc9...","(0, 0, True)",None,None,1566293696,None,87449,None,3327,None,87449_3327,"(1, 1, 9, 0)",87449,"(None, None, iphone, api, None)",post,"(0, 0)",None,Пекло.Пушка.Вышка.\nЧёртов гений Тарантино. Лю...,None,"(366,)"
8,"[(None, None, None, None, None, None, (02804b0...","(0, 0, True)",None,None,1571221102,None,87449,None,3329,None,87449_3329,"(1, 1, 0, 0)",87449,"(None, None, iphone, api, None)",post,"(0, 0)",None,,None,"(139,)"
9,"[(None, None, None, None, None, None, (a88e65c...","(1, 1, True)",None,None,1559647730,None,94494,None,11752,None,94494_11752,"(1, 1, 17, 0)",94494,"(None, None, None, vk, None)",post,"(0, 0)",None,"Тот неловкий момент, когда прилетел из унылого...",None,"(594,)"


## Task 3

Get reposts of the original posts of the itmo group (posts.json) from user posts (the result should be similar to (group_post_id, Array (user_post_ids)))

In [12]:
group_reposts = followers_posts \
    .select(explode("copy_history") \
    .alias("original_post"), "owner_id", "id") \
    .withColumnRenamed("id", "user_post_id") \
    .filter("original_post.owner_id == -94") \
    .select("original_post.id", "owner_id", "user_post_id") \
    .withColumnRenamed("id", "group_post_id") \
    .groupBy("group_post_id") \
    .agg(collect_list("user_post_id").alias("user_post_ids"))
  
save_as_json(group_reposts, "task3_group_reposts.json")

group_reposts.limit(10).toPandas()

,group_post_id,user_post_ids
0,41424,"[6941, 4125, 529]"
1,39407,"[3536, 9214]"
2,42388,[11053]
3,39719,[1160]
4,40011,"[4770, 2559, 19267]"
5,38963,"[4868, 3720, 393, 6403, 814]"
6,39259,"[14840, 10683, 3560, 1751, 1205, 822, 10810, 1..."
7,40084,[536]
8,41506,"[12823, 135, 670, 397, 398]"
9,42009,"[1509, 120]"
